# Retail Sales (Istanbul malls). Revenue predict with CatBoost Regressor

https://www.kaggle.com/datasets/mehmettahiraslan/customer-shopping-dataset

## Import libraries

In [1]:
import pandas as pd
import warnings
import keyring
import requests
from dateutil.relativedelta import relativedelta
import os
import numpy as np
import datetime
import matplotlib.pyplot as plt
from datetime import date, timedelta
warnings.filterwarnings("ignore")
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, r2_score, accuracy_score, precision_score, f1_score, roc_auc_score, mean_squared_error
import seaborn as sns

## Load data

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
df = pd.read_csv('/workspaces/dbt_ml_retail/Forecast/customer_shopping_data.csv')

https://drive.google.com/file/d/1syhtZtr0n57iHuc2MYDrorM_ihu5AqRz/view?usp=drive_link

In [4]:
!gdown 1syhtZtr0n57iHuc2MYDrorM_ihu5AqRz #Other way to download from gdrive

Downloading...
From: https://drive.google.com/uc?id=1syhtZtr0n57iHuc2MYDrorM_ihu5AqRz
To: /content/customer_shopping_data.csv
100% 7.54M/7.54M [00:00<00:00, 37.0MB/s]


In [5]:
df = pd.read_csv('/content/customer_shopping_data.csv')

In [6]:
df

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500.400,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1800.510,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300.080,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3000.850,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,60.600,Cash,24/10/2021,Kanyon
...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,58.650,Credit Card,21/09/2022,Kanyon
99453,I325143,C569580,Male,27,Food & Beverage,2,10.460,Cash,22/09/2021,Forum Istanbul
99454,I824010,C103292,Male,63,Food & Beverage,2,10.460,Debit Card,28/03/2021,Metrocity
99455,I702964,C800631,Male,56,Technology,4,4200.000,Cash,16/03/2021,Istinye Park


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   invoice_no      99457 non-null  object 
 1   customer_id     99457 non-null  object 
 2   gender          99457 non-null  object 
 3   age             99457 non-null  int64  
 4   category        99457 non-null  object 
 5   quantity        99457 non-null  int64  
 6   price           99457 non-null  float64
 7   payment_method  99457 non-null  object 
 8   invoice_date    99457 non-null  object 
 9   shopping_mall   99457 non-null  object 
dtypes: float64(1), int64(2), object(7)
memory usage: 7.6+ MB


## Key metric - Revenue

In [10]:
df['revenue'] = df['quantity']*df['price']

In [11]:
df

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall,revenue
0,I138884,C241288,Female,28,Clothing,5,1500.400,Credit Card,5/8/2022,Kanyon,7502.000
1,I317333,C111565,Male,21,Shoes,3,1800.510,Debit Card,12/12/2021,Forum Istanbul,5401.530
2,I127801,C266599,Male,20,Clothing,1,300.080,Cash,9/11/2021,Metrocity,300.080
3,I173702,C988172,Female,66,Shoes,5,3000.850,Credit Card,16/05/2021,Metropol AVM,15004.250
4,I337046,C189076,Female,53,Books,4,60.600,Cash,24/10/2021,Kanyon,242.400
...,...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,58.650,Credit Card,21/09/2022,Kanyon,293.250
99453,I325143,C569580,Male,27,Food & Beverage,2,10.460,Cash,22/09/2021,Forum Istanbul,20.920
99454,I824010,C103292,Male,63,Food & Beverage,2,10.460,Debit Card,28/03/2021,Metrocity,20.920
99455,I702964,C800631,Male,56,Technology,4,4200.000,Cash,16/03/2021,Istinye Park,16800.000


## EDA

In [12]:
def highlight_max(s):
    is_max = s == s.max()
    return ['color: green' if cell else '' for cell in is_max]

In [13]:
df['invoice_date'].max()

'9/9/2022'

In [14]:
df['invoice_date'].min()

'1/1/2021'

In [15]:
df_gender = df.groupby('gender')['revenue'].agg(['count','mean','sum'])
df_gender[['count','mean','sum']].style.apply(highlight_max)

,count,mean,sum
gender,,,
Female,59482,2525.253623,150207136.020000
Male,39975,2534.050237,101298658.230000



Women predominate in the frame

In [16]:
df_payment_method = df.groupby('payment_method')['revenue'].agg(['count','mean','sum'])
df_payment_method[['count','mean','sum']].style.apply(highlight_max)

,count,mean,sum
payment_method,,,
Cash,44447,2538.579500,112832243.020000
Credit Card,34931,2521.460129,88077123.770000
Debit Card,20079,2519.867895,50596427.460000


Cash is used more for purchases

In [17]:
df_category = df.groupby('category')['revenue'].agg(['count','mean','sum'])
df_category[['count','mean','sum']].style.apply(highlight_max)

,count,mean,sum
category,,,
Books,4981,167.547260,834552.900000
Clothing,34487,3305.500364,113996791.040000
Cosmetics,15097,449.947864,6792862.900000
Food & Beverage,14776,57.494251,849535.050000
Shoes,10034,6632.793649,66553451.470000
Souvenir,4999,127.190368,635824.650000
Technology,4996,11581.735388,57862350.000000
Toys,10087,394.609521,3980426.240000


Clothing, shoes & technology are most popular categories per revenue

In [18]:
df['age'].min()

18

In [19]:
df['age'].max()

69

Bins of age group:

In [20]:
df['age_group'] = pd.cut(df['age'],[15,20,25,30,35,40,45,50,55,60,65,np.inf])

In [21]:
df_age_group = df.groupby('age_group')['revenue'].agg(['count','mean','sum'])
df_age_group[['count','mean','sum']].style.apply(highlight_max)

,count,mean,sum
age_group,,,
"(15.0, 20.0]",5624,2428.677585,13658882.740000
"(20.0, 25.0]",9735,2512.520658,24459388.610000
"(25.0, 30.0]",9665,2579.320306,24929130.760000
"(30.0, 35.0]",9394,2443.105048,22950528.820000
"(35.0, 40.0]",9872,2574.947782,25419884.500000
"(40.0, 45.0]",9564,2589.329913,24764351.290000
"(45.0, 50.0]",9502,2527.414785,24015495.290000
"(50.0, 55.0]",9514,2544.066360,24204247.350000
"(55.0, 60.0]",9418,2504.185096,23584415.230000


* Age group between 15 and 20/ 65+ is the least active.
* The sample is balanced by age.
* Age group between 35 and 45 is the aim for us.

In [23]:
df_shopping_mall = df.groupby('shopping_mall')['revenue'].agg(['count','mean','sum'])
df_shopping_mall[['count','mean','sum']].style.apply(highlight_max)

,count,mean,sum
shopping_mall,,,
Cevahir AVM,4991,2533.588099,12645138.200000
Emaar Square Mall,4811,2578.694718,12406100.290000
Forum Istanbul,4947,2487.148017,12303921.240000
Istinye Park,9781,2517.005181,24618827.680000
Kanyon,19823,2550.281547,50554231.100000
Mall of Istanbul,19943,2550.894132,50872481.680000
Metrocity,15011,2485.030133,37302787.330000
Metropol AVM,10161,2497.777108,25379913.190000
Viaport Outlet,4914,2548.095181,12521339.720000


The most popular mall is Mall of Istanbul

## Feature engineering

Lets create features for our model

In [62]:
start_forecast = "2023-03-08"
end_forecast = "2023-12-31"

In [25]:
df.columns

Index(['invoice_no', 'customer_id', 'gender', 'age', 'category', 'quantity',
       'price', 'payment_method', 'invoice_date', 'shopping_mall', 'revenue',
       'age_group'],
      dtype='object')

In [26]:
df_model = df[['gender'#,'age'
,'age_group','category','payment_method','invoice_date','shopping_mall','revenue']]
df_model.head()

,gender,age_group,category,payment_method,invoice_date,shopping_mall,revenue
0,Female,"(25.0, 30.0]",Clothing,Credit Card,2022-08-05,Kanyon,7502.000
1,Male,"(20.0, 25.0]",Shoes,Debit Card,2021-12-12,Forum Istanbul,5401.530
2,Male,"(15.0, 20.0]",Clothing,Cash,2021-11-09,Metrocity,300.080
3,Female,"(65.0, inf]",Shoes,Credit Card,2021-05-16,Metropol AVM,15004.250
4,Female,"(50.0, 55.0]",Books,Cash,2021-10-24,Kanyon,242.400


In [27]:
 df_model['invoice_date'] = pd.to_datetime(df_model['invoice_date'], format='%d/%m/%Y')

In [28]:
df_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   gender          99457 non-null  object        
 1   age_group       99457 non-null  category      
 2   category        99457 non-null  object        
 3   payment_method  99457 non-null  object        
 4   invoice_date    99457 non-null  datetime64[ns]
 5   shopping_mall   99457 non-null  object        
 6   revenue         99457 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(1), object(4)
memory usage: 4.6+ MB


In [29]:
df_model.head()

,gender,age_group,category,payment_method,invoice_date,shopping_mall,revenue
0,Female,"(25.0, 30.0]",Clothing,Credit Card,2022-08-05,Kanyon,7502.000
1,Male,"(20.0, 25.0]",Shoes,Debit Card,2021-12-12,Forum Istanbul,5401.530
2,Male,"(15.0, 20.0]",Clothing,Cash,2021-11-09,Metrocity,300.080
3,Female,"(65.0, inf]",Shoes,Credit Card,2021-05-16,Metropol AVM,15004.250
4,Female,"(50.0, 55.0]",Books,Cash,2021-10-24,Kanyon,242.400


In [30]:
df_model.sort_values(by='invoice_date',ascending=True)

,gender,age_group,category,payment_method,invoice_date,shopping_mall,revenue
36199,Female,"(65.0, inf]",Food & Beverage,Credit Card,2021-01-01,Metrocity,130.750
78539,Female,"(40.0, 45.0]",Clothing,Credit Card,2021-01-01,Mall of Istanbul,4801.280
27812,Male,"(50.0, 55.0]",Clothing,Cash,2021-01-01,Metrocity,2700.720
27856,Male,"(55.0, 60.0]",Food & Beverage,Credit Card,2021-01-01,Mall of Istanbul,83.680
5323,Male,"(30.0, 35.0]",Clothing,Cash,2021-01-01,Istinye Park,300.080
...,...,...,...,...,...,...,...
20863,Female,"(65.0, inf]",Clothing,Cash,2023-03-08,Mall of Istanbul,7502.000
20879,Female,"(35.0, 40.0]",Food & Beverage,Cash,2023-03-08,Kanyon,47.070
61644,Female,"(55.0, 60.0]",Cosmetics,Credit Card,2023-03-08,Kanyon,1016.500
27871,Female,"(45.0, 50.0]",Shoes,Credit Card,2023-03-08,Metrocity,5401.530


In [31]:
df_model.columns

Index(['gender', 'age_group', 'category', 'payment_method', 'invoice_date',
       'shopping_mall', 'revenue'],
      dtype='object')

In [63]:
gender = pd.DataFrame({"gender": df_model.gender.drop_duplicates()})
age_group = pd.DataFrame({"age_group": df_model.age_group.drop_duplicates()})
category = pd.DataFrame({"category": df_model.category.drop_duplicates()})
payment_method = pd.DataFrame({"payment_method": df_model.payment_method.drop_duplicates()})
shopping_mall = pd.DataFrame({"shopping_mall": df_model.shopping_mall.drop_duplicates()})
dates = pd.DataFrame({"invoice_date": pd.date_range(start=min(df_model.invoice_date), end=end_forecast, freq="D")})

gender["key"], age_group ["key"], dates["key"], category["key"], payment_method["key"], shopping_mall["key"] = 0, 0, 0, 0, 0, 0

df_all = pd.merge(pd.merge(pd.merge(pd.merge(pd.merge(gender, age_group , on="key"), dates, on="key"), category, on="key"), payment_method, on="key"), shopping_mall, on="key")
df_all = df_all.drop("key", axis=1)

df = pd.merge(df_all, df_model, on=["gender", "age_group", "category", "payment_method", "shopping_mall", "invoice_date"], how="left")
df["revenue"] = np.where(df["revenue"].isnull(), 0, df["revenue"])

df["year"] = df["invoice_date"].dt.year
df["day"] = df['invoice_date'].dt.day
df["month"] = df["invoice_date"].dt.month
df["weekday"] = df["invoice_date"].dt.weekday
df["weeknumber"] = df["invoice_date"].dt.weekofyear

df["is1jan"] = np.where((df["month"] == 1) & (df["invoice_date"].dt.day == 1), 1, 0)
df["wn"] = np.where(df["weekday"] <= 4, 1, 0)
df

,gender,age_group,invoice_date,category,payment_method,shopping_mall,revenue,year,day,month,weekday,weeknumber,is1jan,wn
0,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Kanyon,0.000,2021,1,1,4,53,1,1
1,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Forum Istanbul,0.000,2021,1,1,4,53,1,1
2,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Metrocity,0.000,2021,1,1,4,53,1,1
3,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Metropol AVM,0.000,2021,1,1,4,53,1,1
4,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Istinye Park,0.000,2021,1,1,4,53,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5784352,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Mall of Istanbul,0.000,2023,31,12,6,52,0,0
5784353,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Emaar Square Mall,0.000,2023,31,12,6,52,0,0
5784354,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Cevahir AVM,0.000,2023,31,12,6,52,0,0
5784355,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Viaport Outlet,0.000,2023,31,12,6,52,0,0


In [33]:
from sklearn.preprocessing import LabelEncoder

In [34]:
le=LabelEncoder()

In [64]:
df['gender_ohe']=le.fit_transform(df['gender']).astype('str')
df['category_ohe']=le.fit_transform(df['category']).astype('str')
df['payment_method_ohe']=le.fit_transform(df['payment_method']).astype('str')
df['shopping_mall_ohe']=le.fit_transform(df['shopping_mall']).astype('str')
df['age_group_ohe']=le.fit_transform(df['age_group']).astype('str')

In [65]:
df.head()

,gender,age_group,invoice_date,category,payment_method,shopping_mall,revenue,year,day,month,weekday,weeknumber,is1jan,wn,gender_ohe,category_ohe,payment_method_ohe,shopping_mall_ohe,age_group_ohe
0,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Kanyon,0.000,2021,1,1,4,53,1,1,0,1,1,4,2
1,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Forum Istanbul,0.000,2021,1,1,4,53,1,1,0,1,1,2,2
2,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Metrocity,0.000,2021,1,1,4,53,1,1,0,1,1,6,2
3,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Metropol AVM,0.000,2021,1,1,4,53,1,1,0,1,1,7,2
4,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Istinye Park,0.000,2021,1,1,4,53,1,1,0,1,1,3,2


In [66]:
features_base_model = ["gender_ohe", "age_group_ohe", "category_ohe","payment_method_ohe","shopping_mall_ohe", "day","month","weekday","is1jan","wn"]

## Revenue forecast

In [39]:
pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.4 MB/s eta 0:00:00


In [40]:
from catboost import CatBoostRegressor

In [91]:
df.head()

,gender,age_group,invoice_date,category,payment_method,shopping_mall,revenue,year,day,month,weekday,weeknumber,is1jan,wn,gender_ohe,category_ohe,payment_method_ohe,shopping_mall_ohe,age_group_ohe
0,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Kanyon,0.000,2021,1,1,4,53,1,1,0,1,1,4,2
1,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Forum Istanbul,0.000,2021,1,1,4,53,1,1,0,1,1,2,2
2,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Metrocity,0.000,2021,1,1,4,53,1,1,0,1,1,6,2
3,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Metropol AVM,0.000,2021,1,1,4,53,1,1,0,1,1,7,2
4,Female,"(25.0, 30.0]",2021-01-01,Clothing,Credit Card,Istinye Park,0.000,2021,1,1,4,53,1,1,0,1,1,3,2


In [68]:
X = features_base_model

y = ['revenue']

In [69]:
parameters = {'eval_metric': 'MAPE',
              'random_seed':42,
              'verbose':100}

In [70]:
model = CatBoostRegressor(**parameters)

In [71]:
df_model['invoice_date'].max()

Timestamp('2023-03-08 00:00:00')

In [75]:
start_forecast

'2023-03-08'

In [76]:
train_date = '2023-02-01'

### First iteration

In [78]:
train_ = df[df['invoice_date']<train_date]
test_ = df[(df['invoice_date']>=train_date )&(df['invoice_date']<start_forecast )]

In [79]:
model.fit(train_[X],train_[y])

Learning rate set to 0.151935
0:	learn: 57.9329428	total: 629ms	remaining: 10m 28s
100:	learn: 57.8401998	total: 1m 6s	remaining: 9m 53s
200:	learn: 58.1122069	total: 2m 10s	remaining: 8m 39s
300:	learn: 58.4887015	total: 3m 14s	remaining: 7m 31s
400:	learn: 58.8982404	total: 4m 24s	remaining: 6m 34s
500:	learn: 59.2484527	total: 5m 25s	remaining: 5m 24s
600:	learn: 59.5923332	total: 6m 25s	remaining: 4m 16s
700:	learn: 59.8609910	total: 7m 24s	remaining: 3m 9s
800:	learn: 60.0693041	total: 8m 27s	remaining: 2m 6s
900:	learn: 60.3369294	total: 9m 27s	remaining: 1m 2s
999:	learn: 60.5652028	total: 10m 27s	remaining: 0us


In [80]:
model.predict(test_[X])

array([597.87793851, 184.95568743, 433.04386756, ...,  -9.26364378,
        -3.45102448,  -3.00890458])

In [81]:
test_['predict_revenue']=model.predict(test_[X])

In [83]:
test_["predict_revenue"] = np.where((test_["predict_revenue"] < 0), 0, test_["predict_revenue"])

In [84]:
test_

,gender,age_group,invoice_date,category,payment_method,shopping_mall,revenue,year,day,month,weekday,weeknumber,is1jan,wn,gender_ohe,category_ohe,payment_method_ohe,shopping_mall_ohe,age_group_ohe,predict_revenue
182830,Female,"(25.0, 30.0]",2023-02-01,Clothing,Credit Card,Kanyon,0.000,2023,1,2,2,5,0,1,0,1,1,4,2,597.878
182831,Female,"(25.0, 30.0]",2023-02-01,Clothing,Credit Card,Forum Istanbul,0.000,2023,1,2,2,5,0,1,0,1,1,2,2,184.956
182832,Female,"(25.0, 30.0]",2023-02-01,Clothing,Credit Card,Metrocity,0.000,2023,1,2,2,5,0,1,0,1,1,6,2,433.044
182833,Female,"(25.0, 30.0]",2023-02-01,Clothing,Credit Card,Metropol AVM,0.000,2023,1,2,2,5,0,1,0,1,1,7,2,288.954
182834,Female,"(25.0, 30.0]",2023-02-01,Clothing,Credit Card,Istinye Park,7502.000,2023,1,2,2,5,0,1,0,1,1,3,2,327.073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5712592,Male,"(40.0, 45.0]",2023-03-07,Souvenir,Cash,Mall of Istanbul,0.000,2023,7,3,1,10,0,1,1,5,0,5,5,0.000
5712593,Male,"(40.0, 45.0]",2023-03-07,Souvenir,Cash,Emaar Square Mall,0.000,2023,7,3,1,10,0,1,1,5,0,1,5,0.000
5712594,Male,"(40.0, 45.0]",2023-03-07,Souvenir,Cash,Cevahir AVM,0.000,2023,7,3,1,10,0,1,1,5,0,0,5,0.000
5712595,Male,"(40.0, 45.0]",2023-03-07,Souvenir,Cash,Viaport Outlet,0.000,2023,7,3,1,10,0,1,1,5,0,8,5,0.000


### Evaluate of error

In [85]:
def mean_absolute_error_day(y):
    y_piv = y.groupby(['invoice_date'])['predict_revenue', 'revenue'].sum().reset_index()
    y_piv['mae_revenue'] = np.abs(y_piv['predict_revenue'] - y_piv['revenue'])
    return np.mean(y_piv['mae_revenue'])/np.mean(y_piv['revenue'])

In [86]:
mean_absolute_error_day(test_)

0.16101043056067738

Error - 16%

### Full train data

In [93]:
df_model['invoice_date'].max()

Timestamp('2023-03-08 00:00:00')

In [94]:
start_forecast

'2023-03-08'

In [95]:
train = df[df['invoice_date']<=start_forecast]
df_predict= df[df['invoice_date']>start_forecast]

In [96]:
model.fit(train[X],train[y])

Learning rate set to 0.153049
0:	learn: 58.0895912	total: 661ms	remaining: 11m
100:	learn: 57.9309548	total: 1m	remaining: 9m 1s
200:	learn: 58.1415086	total: 1m 59s	remaining: 7m 53s
300:	learn: 58.5900989	total: 2m 58s	remaining: 6m 55s
400:	learn: 58.9626165	total: 3m 57s	remaining: 5m 54s
500:	learn: 59.2970326	total: 5m 3s	remaining: 5m 2s
600:	learn: 59.6202976	total: 6m 3s	remaining: 4m 1s
700:	learn: 59.8951767	total: 7m 4s	remaining: 3m
800:	learn: 60.1790677	total: 8m 4s	remaining: 2m
900:	learn: 60.4554980	total: 9m 2s	remaining: 59.6s
999:	learn: 60.6841262	total: 10m 3s	remaining: 0us


In [97]:
model.predict(df_predict[X])

array([553.07989826, 150.59662893, 430.81417718, ..., -11.86572815,
       -13.75123474, -25.75886306])

In [98]:
df_predict['revenue']=model.predict(df_predict[X])

In [99]:
df_predict["revenue"] = np.where((df_predict["revenue"] < 0), 0, df_predict["revenue"])

In [100]:
df_predict

,gender,age_group,invoice_date,category,payment_method,shopping_mall,revenue,year,day,month,weekday,weeknumber,is1jan,wn,gender_ohe,category_ohe,payment_method_ohe,shopping_mall_ohe,age_group_ohe
191480,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Kanyon,553.080,2023,9,3,3,10,0,1,0,1,1,4,2
191481,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Forum Istanbul,150.597,2023,9,3,3,10,0,1,0,1,1,2,2
191482,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Metrocity,430.814,2023,9,3,3,10,0,1,0,1,1,6,2
191483,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Metropol AVM,284.948,2023,9,3,3,10,0,1,0,1,1,7,2
191484,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Istinye Park,285.615,2023,9,3,3,10,0,1,0,1,1,3,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5784352,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Mall of Istanbul,10.947,2023,31,12,6,52,0,0,1,5,0,5,5
5784353,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Emaar Square Mall,9.754,2023,31,12,6,52,0,0,1,5,0,1,5
5784354,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Cevahir AVM,0.000,2023,31,12,6,52,0,0,1,5,0,0,5
5784355,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Viaport Outlet,0.000,2023,31,12,6,52,0,0,1,5,0,8,5


In [102]:
df_predict_base = [train,df_predict]

df_predict_base = pd.concat(df_predict_base)

df_predict_base_revenue_per_year = df_predict_base.groupby('year')['revenue'].sum().reset_index()
df_predict_base_revenue_per_year
#train_test_revenue_per_year
#df['lagged_values'] = df.groupby(['group'])['values'].shift(1)

,year,revenue
0,2021,114560570.590
1,2022,115436814.080
2,2023,118987438.270


## Lets create the final dataset with predicted values

In [103]:
df_predict.columns

Index(['gender', 'age_group', 'invoice_date', 'category', 'payment_method',
       'shopping_mall', 'revenue', 'year', 'day', 'month', 'weekday',
       'weeknumber', 'is1jan', 'wn', 'gender_ohe', 'category_ohe',
       'payment_method_ohe', 'shopping_mall_ohe', 'age_group_ohe'],
      dtype='object')

In [107]:
df_predict_share = (df_predict[['gender', 'age_group', 'invoice_date', 'category', 'payment_method',
       'shopping_mall', 'revenue']])

df_predict_share = (df_predict_share[df_predict_share['revenue'] != 0 ])

df_predict_share

,gender,age_group,invoice_date,category,payment_method,shopping_mall,revenue
191480,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Kanyon,553.080
191481,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Forum Istanbul,150.597
191482,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Metrocity,430.814
191483,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Metropol AVM,284.948
191484,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Istinye Park,285.615
...,...,...,...,...,...,...,...
5784347,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Kanyon,18.969
5784348,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Forum Istanbul,16.776
5784351,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Istinye Park,26.701
5784352,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Mall of Istanbul,10.947


In [112]:
df_predict_share['month_year'] =  df_predict_share['invoice_date'].dt.strftime('%Y-%m-01')
df_predict_share['month_year'] = pd.to_datetime(df_predict_share['month_year'], format='%Y-%m-%d')

In [130]:
df_predict_share

,gender,age_group,invoice_date,category,payment_method,shopping_mall,revenue,month_year
191480,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Kanyon,553.080,2023-03-01
191481,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Forum Istanbul,150.597,2023-03-01
191482,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Metrocity,430.814,2023-03-01
191483,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Metropol AVM,284.948,2023-03-01
191484,Female,"(25.0, 30.0]",2023-03-09,Clothing,Credit Card,Istinye Park,285.615,2023-03-01
...,...,...,...,...,...,...,...,...
5784347,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Kanyon,18.969,2023-12-01
5784348,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Forum Istanbul,16.776,2023-12-01
5784351,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Istinye Park,26.701,2023-12-01
5784352,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Mall of Istanbul,10.947,2023-12-01


In [131]:
df_predict_share.to_csv('df_predict.csv')

In [126]:
df_model['month_year'] =  df_model['invoice_date'].dt.strftime('%Y-%m-01')
df_model['month_year'] = pd.to_datetime(df_model['month_year'], format='%Y-%m-%d')

In [127]:
df_model = df_model[['gender','age_group','invoice_date','category','payment_method','shopping_mall','revenue','month_year']]

In [128]:
df_model

,gender,age_group,invoice_date,category,payment_method,shopping_mall,revenue,month_year
0,Female,"(25.0, 30.0]",2022-08-05,Clothing,Credit Card,Kanyon,7502.000,2022-08-01
1,Male,"(20.0, 25.0]",2021-12-12,Shoes,Debit Card,Forum Istanbul,5401.530,2021-12-01
2,Male,"(15.0, 20.0]",2021-11-09,Clothing,Cash,Metrocity,300.080,2021-11-01
3,Female,"(65.0, inf]",2021-05-16,Shoes,Credit Card,Metropol AVM,15004.250,2021-05-01
4,Female,"(50.0, 55.0]",2021-10-24,Books,Cash,Kanyon,242.400,2021-10-01
...,...,...,...,...,...,...,...,...
99452,Female,"(40.0, 45.0]",2022-09-21,Souvenir,Credit Card,Kanyon,293.250,2022-09-01
99453,Male,"(25.0, 30.0]",2021-09-22,Food & Beverage,Cash,Forum Istanbul,20.920,2021-09-01
99454,Male,"(60.0, 65.0]",2021-03-28,Food & Beverage,Debit Card,Metrocity,20.920,2021-03-01
99455,Male,"(55.0, 60.0]",2021-03-16,Technology,Cash,Istinye Park,16800.000,2021-03-01


In [132]:
df_model.to_csv('df_base.csv')

In [133]:
df_full_data = [df_model,df_predict_share]
df_full_data = pd.concat(df_full_data)
df_full_data

,gender,age_group,invoice_date,category,payment_method,shopping_mall,revenue,month_year
0,Female,"(25.0, 30.0]",2022-08-05,Clothing,Credit Card,Kanyon,7502.000,2022-08-01
1,Male,"(20.0, 25.0]",2021-12-12,Shoes,Debit Card,Forum Istanbul,5401.530,2021-12-01
2,Male,"(15.0, 20.0]",2021-11-09,Clothing,Cash,Metrocity,300.080,2021-11-01
3,Female,"(65.0, inf]",2021-05-16,Shoes,Credit Card,Metropol AVM,15004.250,2021-05-01
4,Female,"(50.0, 55.0]",2021-10-24,Books,Cash,Kanyon,242.400,2021-10-01
...,...,...,...,...,...,...,...,...
5784347,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Kanyon,18.969,2023-12-01
5784348,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Forum Istanbul,16.776,2023-12-01
5784351,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Istinye Park,26.701,2023-12-01
5784352,Male,"(40.0, 45.0]",2023-12-31,Souvenir,Cash,Mall of Istanbul,10.947,2023-12-01


In [134]:
df_full_data.to_csv('df_full_data.csv')